In [2]:
#import the three relevant text files for analysis

flags = [
    "ftrapv_flag",
    "fwrapv_flag",
    "Wnarrowing"
]

file_texts = {}

for flag in flags:
    filename = flag + "_build_log.txt"
    
    with open(filename) as opened_file:
        file_texts[flag] = opened_file.readlines()

file_texts

{'ftrapv_flag': ['Starting local Bazel server and connecting to it...\n',
  'INFO: Options provided by the client:\n',
  "  Inherited 'common' options: --isatty=0 --terminal_columns=80\n",
  "INFO: Reading rc options for 'build' from /usr/local/google/home/taregaskin/Desktop/tensorflow/.bazelrc:\n",
  "  Inherited 'common' options: --experimental_repo_remote_exec\n",
  "INFO: Reading rc options for 'build' from /usr/local/google/home/taregaskin/Desktop/tensorflow/.bazelrc:\n",
  "  'build' options: --apple_platform_type=macos --define framework_shared_object=true --define open_source_build=true --java_toolchain=//third_party/toolchains/java:tf_java_toolchain --host_java_toolchain=//third_party/toolchains/java:tf_java_toolchain --define=use_fast_cpp_protos=true --define=allow_oversize_protos=true --spawn_strategy=standalone -c opt --announce_rc --define=grpc_no_ares=true --noincompatible_remove_legacy_whole_archive --noincompatible_prohibit_aapt1 --enable_platform_specific_config --conf

In [10]:
# Index all flags: The below procedure
# will count the occurences of each unique warning tag
# in each of the 3 files. 

import re

unique_warning_tags = {}

for flag in flags:
    filename = flag + "_build_log.txt"
    
    with open(filename) as opened_file:
        matches_in_file = re.findall(r'(\[-W.+\])', opened_file.read())
        for match in matches_in_file:
            if match in unique_warning_tags:
                if filename not in unique_warning_tags[match]:
                    unique_warning_tags[match][filename] = 1 
                else:
                    unique_warning_tags[match][filename] += 1
            else:
                unique_warning_tags[match] =  { filename: 1 }
        

list(unique_warning_tags.keys()), len(unique_warning_tags)

(['[-Wsign-compare]',
  '[-Wunknown-pragmas]',
  '[-Wclass-memaccess]',
  '[-Wcomment]',
  '[-Wunused-result]',
  '[-Wmaybe-uninitialized]',
  '[-Wrestrict]',
  '[-Wdeprecated-declarations]',
  '[-Wunused-function]',
  '[-Wstringop-truncation]',
  '[-Wignored-attributes]',
  '[-Wunused-but-set-parameter]',
  '[-Winvalid-offsetof]',
  '[-Wcpp]',
  '[-Wint-in-bool-context]',
  '[-Warray-bounds]',
  '[-Wreturn-type]',
  '[-Wreorder]',
  '[-Wattributes]',
  '[-Wstrict-aliasing]',
  '[-Wuninitialized]',
  '[-Wparentheses]',
  '[-Wunused-but-set-variable]',
  '[-Wbool-compare]',
  '[-Wunused-variable]'],
 25)

In [11]:
'''
    See below that there are appparently 3324
    occurences of the '[-Wsign-compare]' tag in 
    all three files.
    
    * It should be noted that for each warning
    generated by the compiler this tag can appear
    multiple times ( that is, the same warning
    gets tagged more than once ). 
    
    * It should also be noted that there is an
    expected overlap in the warnings generated
    between files. This overlap may be total (
    i.e. the three files have all '[-Wsign-compare]'
    warnings in common )

'''
unique_warning_tags['[-Wsign-compare]']

{'ftrapv_flag_build_log.txt': 3324,
 'fwrapv_flag_build_log.txt': 3324,
 'Wnarrowing_build_log.txt': 3324}

In [23]:
# Retrieve all the [-Wsign-compare] flags


# target tag is one of the discovered warning tags strings,
# file texts is a dict< string, list<string> > where 
# * the keys of the dict are  the c_opt_flag used to generate the bazel build log file
# * strings of the lists are the lines of the file


def retrieve_unique_warnings(target_tag, file_texts):
    target_tag_warnings = {}
    for flag in file_texts:
        file_lines = file_texts[flag]

        i=0
        # scan to first line that starts with "INFO:"
        while( (i < len(file_lines)) and (file_lines[i][:5] != "INFO:" ) ):
            i+=1

        while ( i < len(file_lines) ):
            # from here we want to add lines to the compiler_warning_string,
            # until we arrive at the next "INFO:", our indicator that we've reached a new warning.
            compiler_warning_string = ""
            compiler_warning_string += file_lines[i] 
            i+= 1

            while( (i < len(file_lines)) and (file_lines[i][:5] != "INFO:" ) ):
                compiler_warning_string += file_lines[i]
                i+=1

            if target_tag in compiler_warning_string:
                #(re.match('\[-Wsign\-compare\]', compiler_warning_string) != None):
                if compiler_warning_string in target_tag_warnings:
                    if flag not in target_tag_warnings[compiler_warning_string]:
                        target_tag_warnings[compiler_warning_string][flag] = 1 
                    else:
                        target_tag_warnings[compiler_warning_string][flag] += 1
                else:
                    target_tag_warnings[compiler_warning_string] =  { flag: 1 }
    return target_tag_warnings
    
sign_compare_warnings = retrieve_unique_warnings('[-Wsign-compare]', file_texts)
array_bounds_warnings = retrieve_unique_warnings('[-Warray-bounds]', file_texts)


len(sign_compare_warnings), len(array_bounds_warnings)

(527, 1)

In [21]:
sign_compare_warnings

{"INFO: From Compiling tensorflow/lite/arena_planner.cc [for host]:\ntensorflow/lite/arena_planner.cc: In member function 'virtual TfLiteStatus tflite::ArenaPlanner::ExecuteAllocations(int, int)':\ntensorflow/lite/arena_planner.cc:200:33: warning: comparison of integer expressions of different signedness: 'size_t' {aka 'long unsigned int'} and 'int' [-Wsign-compare]\n  200 |   for (size_t i = first_node; i <= last_node && i < graph_info_->num_nodes();\n      |                               ~~^~~~~~~~~~~~\nAt global scope:\ncc1plus: warning: unrecognized command line option '-Wno-extern-c-compat'\n": {'ftrapv_flag': 2,
  'fwrapv_flag': 2,
  'Wnarrowing': 2},
 "INFO: From Compiling tensorflow/core/platform/protobuf.cc:\ntensorflow/core/platform/protobuf.cc: In member function 'virtual bool tensorflow::TStringOutputStream::Next(void**, int*)':\ntensorflow/core/platform/protobuf.cc:29:16: warning: comparison of integer expressions of different signedness: 'int' and 'size_t' {aka 'long unsi

In [29]:
array_bounds_warnings

{"INFO: From Compiling tensorflow/core/kernels/sparse/sparse_matrix.cc [for host]:\nIn file included from /usr/include/string.h:495,\n                 from external/com_google_protobuf/src/google/protobuf/stubs/port.h:38,\n                 from external/com_google_protobuf/src/google/protobuf/stubs/common.h:46,\n                 from external/com_google_protobuf/src/google/protobuf/io/coded_stream.h:135,\n                 from bazel-out/host/bin/tensorflow/core/framework/tensor.pb.h:23,\n                 from ./tensorflow/core/framework/variant_op_registry.h:25,\n                 from tensorflow/core/kernels/sparse/sparse_matrix.cc:22:\nIn function 'void* memcpy(void*, const void*, size_t)',\n    inlined from 'tensorflow::TensorShapeRep::TensorShapeRep(tensorflow::TensorShapeRep&&)' at ./tensorflow/core/framework/tensor_shape.h:519:9,\n    inlined from 'tensorflow::TensorShapeBase<tensorflow::TensorShape>::TensorShapeBase(tensorflow::TensorShapeBase<tensorflow::TensorShape>&&)' at ./te

In [27]:
# process data into a csv report

import pandas as pd

# !filename argument should not include '.csv' extension 
def process_warnings_to_csv_report(warning_object, filename):
    processed_rows = []
    row_index = 0
    for warning_text in warning_object:
        processed_rows.append([row_index, warning_text, 
                                warning_object[warning_text].get('ftrapv_flag', 0),
                                warning_object[warning_text].get('fwrapv_flag', 0),
                                warning_object[warning_text].get('Wnarrowing', 0)
                                ])
        row_index += 1

    column_headers=['warning_id','warning_text','ftrapv_log_count', 'frwapv_log_count','Wnarrowing_log_count']
    df = pd.DataFrame(data=processed_rows, columns=column_headers)
    
    filename = filename.replace(" ", "_") + ".csv"
    df.to_csv(filename)
    
    return df

In [30]:
sign_compare_warnings_df = process_warnings_to_csv_report(sign_compare_warnings, "Wsign-compare_warning_index")
array_bounds_warning_df = process_to_csv_report(array_bounds_warnings, "Warray-bounds_warning_index")

sign_compare_warnings_df

,warning_id,warning_text,ftrapv_log_count,frwapv_log_count,Wnarrowing_log_count
0,0,INFO: From Compiling tensorflow/lite/arena_pla...,2,2,2
1,1,INFO: From Compiling tensorflow/core/platform/...,2,1,2
2,2,INFO: From Compiling tensorflow/core/platform/...,2,2,2
3,3,INFO: From Compiling tensorflow/core/platform/...,2,0,2
4,4,INFO: From Compiling tensorflow/core/platform/...,2,2,2
...,...,...,...,...,...
522,522,INFO: From Compiling tensorflow/core/framework...,0,0,1
523,523,INFO: From Compiling tensorflow/lite/experimen...,0,0,1
524,524,INFO: From Compiling tensorflow/lite/toco/grap...,0,0,1
525,525,INFO: From Compiling tensorflow/compiler/xla/s...,0,0,1


In [ ]:
'''
    Inisghts:
        * The are 527 unique sign compare warnings, 
        * There is 1 unique -Warray-bounds warning, 
        * There is alot of overlap between the 3 files, 
            and it is likely that the files have all
            [-Wsign-compare] errors in common, except
            1 or 2.
            
'''